# Chapter 13: Decorators & Closures

Master function decorators, closures, and advanced function manipulation



### What are Decorators? (Slide 4)


<p><strong>Decorators</strong> are functions that modify the behavior of other functions or classes.</p>
<p><strong>Key Concepts:</strong></p>
<ul>
<li>Functions are first-class objects in Python</li>
<li>Functions can be passed as arguments</li>
<li>Functions can return other functions</li>
<li>Decorators wrap functions to extend behavior</li>
</ul>
<p><strong>Common Uses:</strong></p>
<ul>
<li>Logging function calls</li>
<li>Timing execution</li>
<li>Access control/authentication</li>
<li>Caching results</li>
<li>Input validation</li>
</ul>


### Functions as First-Class Objects (Slide 5)


In [1]:
# Functions can be assigned to variables
def greet(name):
    return f"Hello, {name}!"

# Assign to variable
say_hello = greet
print(say_hello("Alice"))  # Hello, Alice!

# Pass function as argument
def execute_function(func, arg):
    return func(arg)

result = execute_function(greet, "Bob")
print(result)  # Hello, Bob!

# Return function from function
def get_greeting_function():
    def greet_formal(name):
        return f"Good day, {name}!"
    return greet_formal

formal = get_greeting_function()
print(formal("Charlie"))  # Good day, Charlie!


Hello, Alice!
Hello, Bob!
Good day, Charlie!


> **Note:** Functions can be treated like any other object


### Simple Decorator (Slide 6)


In [2]:
# Basic decorator
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

# Manual decoration
def say_hello():
    print("Hello!")

say_hello = my_decorator(say_hello)
say_hello()
# Output:
# Before function call
# Hello!
# After function call

# Using @ syntax (syntactic sugar)
@my_decorator
def say_goodbye():
    print("Goodbye!")

say_goodbye()  # Same behavior


Before function call
Hello!
After function call
Before function call
Goodbye!
After function call


> **Note:** @ syntax is cleaner than manual decoration


### Decorator with Arguments (Slide 7)


In [3]:
# Decorator that accepts function arguments
def timing_decorator(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end-start:.4f}s")
        return result
    return wrapper

@timing_decorator
def slow_function(n):
    total = 0
    for i in range(n):
        total += i
    return total

result = slow_function(1000000)
# Output: slow_function took 0.0523s
print(result)


slow_function took 0.0262s
499999500000


> **Note:** *args, **kwargs let decorator work with any function


### Preserving Function Metadata (Slide 8)


In [4]:
from functools import wraps

# Without @wraps
def bad_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@bad_decorator
def my_function():
    """This is my function"""
    pass

print(my_function.__name__)  # wrapper (wrong!)
print(my_function.__doc__)   # None (lost!)

# With @wraps
def good_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@good_decorator
def my_function2():
    """This is my function"""
    pass

print(my_function2.__name__)  # my_function2 (correct!)
print(my_function2.__doc__)   # This is my function


wrapper
None
my_function2
This is my function


> **Note:** Always use @wraps to preserve metadata


### Decorator with Parameters (Slide 9)


In [5]:
# Decorator factory - returns decorator
def repeat(times):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")
# Output:
# Hello, Alice!
# Hello, Alice!
# Hello, Alice!

# With arguments
@repeat(times=5)
def say_hi():
    print("Hi!")


Hello, Alice!
Hello, Alice!
Hello, Alice!


> **Note:** Three levels: factory → decorator → wrapper


### Practical Decorator - Caching (Slide 10)


In [6]:
from functools import lru_cache

# Memoization - cache results
@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# First call - calculates
print(fibonacci(100))  # Fast!

# Second call - uses cache
print(fibonacci(100))  # Instant!

# Custom cache decorator
def cache(func):
    cached_results = {}
    @wraps(func)
    def wrapper(*args):
        if args not in cached_results:
            cached_results[args] = func(*args)
        return cached_results[args]
    return wrapper

@cache
def expensive_operation(x, y):
    return x ** y


354224848179261915075
354224848179261915075


> **Note:** lru_cache is built-in memoization


### Class Decorators (Slide 11)


In [7]:
# Decorator for classes
def singleton(cls):
    instances = {}
    @wraps(cls)
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Database initialized")

db1 = Database()  # Database initialized
db2 = Database()  # No output
print(db1 is db2)  # True (same instance)

# Add methods to class
def add_str_method(cls):
    cls.__str__ = lambda self: f"{cls.__name__} instance"
    return cls

@add_str_method
class MyClass:
    pass

obj = MyClass()
print(obj)  # MyClass instance


Database initialized
True
MyClass instance


> **Note:** Class decorators modify class definition


### Chaining Decorators (Slide 12)


In [8]:
# Multiple decorators
def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def exclaim(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result + "!"
    return wrapper

# Applied bottom to top
@uppercase
@exclaim
def greet(name):
    return f"hello, {name}"

print(greet("alice"))  # HELLO, ALICE!

# Equivalent to:
# greet = uppercase(exclaim(greet))


HELLO, ALICE!


> **Note:** Decorators apply from bottom to top


### Closures - Basics (Slide 13)


In [9]:
# Closure - inner function remembers outer scope
def outer_function(x):
    # x is in outer scope
    def inner_function(y):
        # Inner can access x
        return x + y
    return inner_function

# Create closures
add_5 = outer_function(5)
add_10 = outer_function(10)

print(add_5(3))   # 8 (5 + 3)
print(add_10(3))  # 13 (10 + 3)

# Each closure has its own x
print(add_5.__closure__)  # Shows captured variables
print(add_5.__closure__[0].cell_contents)  # 5


8
13
(<cell at 0x00000244813DDE40: int object at 0x00007FFC293A54F8>,)
5


> **Note:** Closures capture and remember variables


### Closures - Counter Example (Slide 14)


In [10]:
# Stateful closure
def make_counter():
    count = 0
    def counter():
        nonlocal count  # Modify outer variable
        count += 1
        return count
    return counter

counter1 = make_counter()
counter2 = make_counter()

print(counter1())  # 1
print(counter1())  # 2
print(counter2())  # 1 (separate state)
print(counter1())  # 3

# Without nonlocal
def broken_counter():
    count = 0
    def counter():
        count += 1  # Error! Can't modify
        return count
    return counter


1
2
1
3


> **Note:** nonlocal allows modifying outer variables


### Decorator Factory Pattern (Slide 15)


In [11]:
# Flexible decorator with optional arguments
def smart_decorator(arg=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if arg:
                print(f"[{arg}] Calling {func.__name__}")
            result = func(*args, **kwargs)
            return result
        return wrapper

    # If called without arguments
    if callable(arg):
        return decorator(arg)
    # If called with arguments
    return decorator

@smart_decorator
def func1():
    print("Function 1")

@smart_decorator("INFO")
def func2():
    print("Function 2")

func1()  # Function 1
func2()  # [INFO] Calling func2\nFunction 2


[<function func1 at 0x00000244813C7ED0>] Calling func1
Function 1
[INFO] Calling func2
Function 2


> **Note:** Support both @decorator and @decorator(args)


### Real-World Example - Authentication (Slide 16)


In [12]:
# Login required decorator
def login_required(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Check if user is authenticated
        if not hasattr(wrapper, 'user') or not wrapper.user:
            raise PermissionError("Login required!")
        return func(*args, **kwargs)
    wrapper.user = None  # Default
    return wrapper

@login_required
def view_profile():
    return "Your profile data"

@login_required
def delete_account():
    return "Account deleted"

# Try without login
try:
    view_profile()
except PermissionError as e:
    print(e)  # Login required!

# Set user
view_profile.user = "alice"
print(view_profile())  # Your profile data


Login required!
Your profile data


> **Note:** Decorators can enforce security policies


### Real-World Example - Retry Logic (Slide 17)


In [13]:
import time
from functools import wraps

def retry(max_attempts=3, delay=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts == max_attempts:
                        raise
                    print(f"Attempt {attempts} failed: {e}")
                    print(f"Retrying in {delay}s...")
                    time.sleep(delay)
        return wrapper
    return decorator

@retry(max_attempts=3, delay=0.5)
def unstable_api_call():
    import random
    if random.random() < 0.7:
        raise ConnectionError("API unavailable")
    return "Success!"

# Will retry up to 3 times
result = unstable_api_call()


Attempt 1 failed: API unavailable
Retrying in 0.5s...


> **Note:** Decorators handle cross-cutting concerns


### Decorator Best Practices (Slide 18)


<p><strong>Do:</strong></p>
<ul>
<li>Always use <code>@wraps</code> to preserve metadata</li>
<li>Use <code>*args, **kwargs</code> for flexibility</li>
<li>Document decorator behavior clearly</li>
<li>Keep decorators focused (single responsibility)</li>
<li>Return the original result unless modifying intentionally</li>
</ul>
<p><strong>Don't:</strong></p>
<ul>
<li>Modify function arguments silently</li>
<li>Hide exceptions unnecessarily</li>
<li>Create overly complex decorators</li>
<li>Forget to return wrapper function</li>
</ul>
<p><strong>Common Patterns:</strong></p>
<ul>
<li>Logging & debugging</li>
<li>Access control</li>
<li>Caching/memoization</li>
<li>Rate limiting</li>
<li>Input validation</li>
</ul>
